In [2]:
import numpy as np
import serial
import array
import time
import binascii
import sys
import numpy
from matplotlib import pyplot as plt
import struct
# Change your usb name here (ex. /dev/ttyUSB0)
#ser = serial.Serial('COM4', 256000, timeout=0.1, xonxoff = False, rtscts = False, dsrdtr = False)
ser = serial.Serial('COM30', 230400, timeout=1, xonxoff = False, rtscts = False, dsrdtr = False)
ser.setRTS(False)
ser.setDTR(False)

In [3]:
import numpy as np
import serial
import array
import time
import binascii
import sys
import numpy
from matplotlib import pyplot as plt
import struct

In [4]:
def CRC_Calc(packet: bytearray):
    crc = 0
    for ik in range(2,packet[2]+2):
        crc += packet[ik]
    crc = ~crc
    crc &= 0xFF
    return bytearray([crc])
    

In [5]:
def Send_Packet(cmd: bytes, data: bytearray, data_length: bytes):
    packet = bytearray([0xFF,0xFF,data_length+2,cmd]) + data
    packet += CRC_Calc(packet)
    ser.flushInput()
    ser.write(packet)
    return 0

In [6]:
cmd = 0x02
data = bytearray([0x05,0x15,0x25])
data_length = 3

Send_Packet(cmd,data,data_length)

0

In [7]:
angle = 0

cmd = 0x03
data = bytearray([0x02,angle&0xFF,(angle>>8)&0xFF])
data_length = 3

Send_Packet(cmd,data,data_length)

0

In [73]:
def robot_set_coord(x: float, y: float, phi: float):
    cmd = 0x06
    data = bytearray(struct.pack("f", x))
    data += bytearray(struct.pack("f", y))
    data += bytearray(struct.pack("f", phi))
    data_length = 12
    Send_Packet(cmd,data,data_length)

In [74]:
def robot_set_speed(dx: float, dy: float, dphi: float):
    cmd = 0x04
    data = bytearray(struct.pack("f", dx))
    data += bytearray(struct.pack("f", dy))
    data += bytearray(struct.pack("f", dphi))
    data_length = 12
    Send_Packet(cmd,data,data_length)

In [75]:
def robot_get_speed():
    cmd = 0x05
    data = bytearray()
    data_length = 0
    Send_Packet(cmd,data,data_length)
    rx_packet = ser.read(17)
    speed = struct.unpack('f', rx_packet[4:8]) 
    speed += struct.unpack('f', rx_packet[8:12])
    speed += struct.unpack('f', rx_packet[12:16])
    return speed

In [76]:
def robot_get_coord():
    cmd = 0x07
    data = bytearray()
    data_length = 0
    Send_Packet(cmd,data,data_length)
    rx_packet = ser.read(17)
    coord = struct.unpack('f', rx_packet[4:8]) 
    coord += struct.unpack('f', rx_packet[8:12])
    coord += struct.unpack('f', rx_packet[12:16])
    return coord

In [77]:
robot_set_coord(0,0,0)

In [86]:
robot_set_speed(0.,0,0
)


In [65]:
ser.flushInput()

In [66]:
speed = robot_get_speed()
print(speed)

SerialException: WriteFile failed (PermissionError(13, 'Устройство не опознает команду.', None, 22))

In [25]:
coord = robot_get_coord()
print(coord)

(4.070012105117371e-10, -1.1687468941090629e-05, 5.436032733996399e-05)


In [ ]:
ser.close()

In [485]:
A = np.array([[np.cos(np.pi/6), 0.5, 1],[-np.cos(np.pi/6),0.5,1],[0,-1,1]])
A_inv = np.array([[1/(2*np.cos(np.pi/6)),-1/(2*np.cos(np.pi/6)),0],[1/3,1/3,-2/3],[1/3,1/3,1/3]])

L = 0.1075
R = 0.03
TicksPerTurn = 4096
Gear = 28

A_gain = np.array([[1,0,0],[0,1,0],[0,0,L]])
A_inv_gain = np.array([[1,0,0],[0,1,0],[0,0,1/L]])

scale = 2*np.pi*R/TicksPerTurn/Gear

A = A.dot(A_gain)
A_inv = A_inv_gain.dot(A_inv)

A_inv = A_inv*scale
A = A/scale

In [487]:
vx = 0.1
vy = 0.0
p = 0

v_in = np.array([[vx],[vy],[p]])

pwm = A.dot(v_in)
print(pwm)

v_out = A_inv.dot(pwm)
print(v_out)

[[ 52692.34029845]
 [-52692.34029845]
 [     0.        ]]
[[0.1]
 [0. ]
 [0. ]]


In [96]:
Vx = 0
Vy = 0.7
W = 0
L = 0.1075
SP = np.array([[Vx],[Vy],[W*L]])
SP = SP*v_to_pwm
V_123 = A.dot(SP)
print(V_123)

[[ 2730.]
 [ 2730.]
 [-5460.]]


In [86]:
SP[2] = SP[2]*L
V_123 = A.dot(SP)
print(V_123)

[[ 4095.]
 [ 4095.]
 [-4095.]]


In [98]:
V1 = 4095
V2 = 4095
V3 = -4095
L = 0.1075
V_123 = np.array([[V1],[V2],[V3]])
SP = A_inv.dot(V_123)
print(SP)
SP[2] = SP[2]/L
print(SP)
v_to_pwm = SP[1]/0.7

[[   0.]
 [5460.]
 [1365.]]
[[    0.       ]
 [ 5460.       ]
 [12697.6744186]]


In [80]:
print(SP)

[[0. ]
 [0.7]
 [0. ]]


In [4]:
high = np.array([[0.7],[0.7],[1.5]])

In [24]:
V0 = np.array([[0],[0],[0]])
V_pos = np.array([[26213],[26213],[26213]])
V_neg = np.array([[-26213],[-26213],[-26213]])
V_max_forw = np.array([[26213],[26213],[26213]])

In [57]:
V_max_forw = np.array([[4096],[4096],[-4096]])

In [55]:
Rot_max = np.array([[500],[0],[38092.8]])

In [6]:
x_t1_10 = np.array([[-49486],[-3651],[45601]])
x_t1_20 = np.array([[-116670],[-6177],[112415]])
x_t1_20_05pi = np.array([[-223155],[-72042],[-2523]])

In [7]:
x_t2_10 = np.array([[-62184],[-2413],[59258]])
x_t2_10_05pi = np.array([[-168492],[-74109],[-49733]])
x_t2_20 = np.array([[-222463],[-80488],[14232]])

In [59]:
high_limitation = A_inv.dot(V_max_forw)
print(high_limitation)
#38092.8 
#4726.784
#5457.7
#0.1347
#0.1075
#5455.872
#12697.6
#k = 7794.103

[[    0.   ]
 [ 5455.872]
 [12697.6  ]]


In [62]:
 = high_limitation[1]

array([27279.36])

In [46]:
high_limitation = A_inv.dot(V_max_forw)
print(high_limitation)
#38092.8 
#4726.784

[[2363.392]
 [4091.904]
 [   0.   ]]


In [56]:
high_limitation = A.dot(Rot_max)
print(high_limitation)

[[4527.976]
 [3661.976]
 [4094.976]]


In [426]:
high_limitation = A_inv.dot(x_t1_10)
print(high_limitation)

[[-26446.795]
 [-48064.887]
 [-23361.6  ]]


In [427]:
high_limitation = A_inv.dot(x_t2_10)
print(high_limitation)

[[-34487.867]
 [-60976.629]
 [-16550.9  ]]


In [430]:
high_limitation = A_inv.dot(x_t1_20_05pi)
print(high_limitation)

[[ -87192.201]
 [ -96620.283]
 [-922932.   ]]


In [431]:
high_limitation = A_inv.dot(x_t2_20)
print(high_limitation)

[[ -81919.575]
 [-110361.195]
 [-895028.9  ]]


In [388]:
low = A.dot(high)

In [392]:
high_back = A_inv.dot(low)

In [393]:
print(high_back)

[[0.6995548]
 [0.6993   ]
 [1.499625 ]]


In [390]:
print(high)

[[0.7]
 [0.7]
 [1.5]]


In [391]:
print(low)

[[ 1.11745]
 [-0.09495]
 [-0.53875]]


In [173]:
L = 0.13417;           # deistance from wheel center to [0;0]
R = 0.03;              # wheel radius
gearhead = 28;         # gear ratio
encoder_ticks = 4096;  #encoder ticks per 1 revolution

In [181]:
#IT IS FOR GETTING TICKS
while(1):
    ticks = get_ticks(0x61)
    ticks += get_ticks(0x62)
    ticks += get_ticks(0x63)
    print(ticks)
    #time.sleep(0.001)

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

...

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(2148335914, 2155130169, 2153559275)
(

KeyboardInterrupt: 

In [236]:
#TRASH
ticks = get_ticks(0x61)
ticks += get_ticks(0x62)
ticks += get_ticks(0x63)
print(ticks)

(2114558719, 2190854943, 2191056476)


In [9]:
maxon_set_local_speed(0,0,4)

In [2]:
DYN_LEFT_BUCKET_1 = 11
DYN_LEFT_BUCKET_2 = 17
DYN_RIGHT_BUCKET_1 = 15
DYN_RIGHT_BUCKET_2 = 14

SHOVEL_DOWN = 0X18
SHOVEL_UP = 0X17

LEFT_STACK_LIFT = 0X19
LEFT_STACK_PUT = 0X21
RIGHT_STACK_LIFT = 0X18
RIGHT_STACK_PUT = 0X20

ALL_DYN_TORQUE_DISABLE = 0X22

MAXON_SET_PWM_BY_ID = 0X68
MAXON_ENABLE_BY_ID = 0X69
MAXON_DISABLE_BY_ID = 0X70
MAXON_GET_SPEED_IN_TICKS_BY_ID = 0x71

SET_LOCAL_SPEED_IN_MS = 0X08
GET_LOCAL_SPEED_IN_MS = 0X09
SET_GLOBAL_CORD_IN_M = 0X0E
GET_GLOBAL_CORD_IN_M = 0X0F
SHOWEL_DOWN = 0x1C
SHOWEL_UP = 0x1D

MOVE_CART_FORWARD = 0X1E
MOVE_CART_BACK = 0X1F

In [645]:
send_cmd(SHOWEL_UP)

In [646]:
send_cmd(SHOWEL_DOWN)

In [686]:
send_cmd(MOVE_CART_BACK)

In [687]:
send_cmd(MOVE_CART_FORWARD)

In [633]:
while(1):
    send_cmd(SHOWEL_UP)
    time.sleep(0.9)
    send_cmd(SHOWEL_DOWN)
    time.sleep(0.9)

KeyboardInterrupt: 

In [514]:
maxon_enable_by_id(0x13)
maxon_enable_by_id(0x12)
maxon_enable_by_id(0x11)

In [515]:
maxon_disable_by_id(0x13)
maxon_disable_by_id(0x12)
maxon_disable_by_id(0x11)

In [304]:
maxon_set_pwm_by_id(0x13,-0.9*32767)

In [373]:
maxon_set_pwm_by_id(0x13,-0.8*32767)
maxon_set_pwm_by_id(0x12,-0.8*32767)
maxon_set_pwm_by_id(0x11,-0.8*32767)

In [506]:
maxon_set_pwm_by_id(0x13,1200)
maxon_set_pwm_by_id(0x12,1200)
maxon_set_pwm_by_id(0x11,1200)


In [512]:
s = 400
maxon_set_pwm_by_id(0x13,s)
maxon_set_pwm_by_id(0x12,s)
maxon_set_pwm_by_id(0x11,s)

In [485]:
send_cmd(LEFT_STACK_LIFT)

In [496]:
send_cmd(LEFT_STACK_PUT)

In [490]:
send_cmd(RIGHT_STACK_LIFT)

In [491]:
send_cmd(RIGHT_STACK_PUT)

In [493]:
send_cmd(ALL_DYN_TORQUE_DISABLE)

In [519]:
ser.flushInput()
speed = maxon_get_speed_in_ticks_by_id(0x11)
print(speed)
speed = maxon_get_speed_in_ticks_by_id(0x12)
print(speed)
speed = maxon_get_speed_in_ticks_by_id(0x13)
print(speed)

(394,)
(401,)
(383,)


In [476]:
while(1):
    send_cmd(LEFT_STACK_LIFT)
    send_cmd(RIGHT_STACK_LIFT)
    time.sleep(0.9)
    send_cmd(LEFT_STACK_PUT)
    send_cmd(RIGHT_STACK_PUT)
    time.sleep(0.9)

KeyboardInterrupt: 

In [455]:
while(1):
    send_cmd(SHOVEL_DOWN)
    time.sleep(0.9)
    send_cmd(SHOVEL_UP)
    time.sleep(0.9)

KeyboardInterrupt: 

In [477]:
ser.flushInput()
ang = get_dyn_angle_by_id(DYN_LEFT_BUCKET_1);
print(ang)
ang = get_dyn_angle_by_id(DYN_LEFT_BUCKET_2);
print(ang)

(202,)
(821,)


In [663]:
ser.flushInput()
ang = get_dyn_angle_by_id(10);
print(ang)

(314,)


In [478]:
ser.flushInput()
ang = get_dyn_angle_by_id(DYN_RIGHT_BUCKET_1);
print(ang)
ang = get_dyn_angle_by_id(DYN_RIGHT_BUCKET_2);
print(ang)

(821,)
(202,)


In [425]:
while(1):
    ang = get_dyn_angle_by_id(DYN_LEFT_BUCKET_1);
    print(ang)

(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(21845,)
(200,)
(820,)
(200,)
(820,)
(200,)
(820,)
(200,)
(820,)
(200,)
(820,)
(20

(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(199,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(199,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(199,)
(200,)
(200,)
(200,)

KeyboardInterrupt: 

In [172]:
#HERE YOU NEED TO MAKE CONSTANTS BE EQUALED TO 0
const1 = 0
const2 = 0
const3 = 0

In [164]:
#IT IS FOR GETTING INTEGER VALUE OF TICKS MINUS ZERO OFFSET AND MINUS CONST
maxon1 = int(get_ticks(0x61)[0])-2147483648 - const1
maxon2 = int(get_ticks(0x62)[0])-2147483648 - const2
maxon3 = int(get_ticks(0x63)[0])-2147483648 - const3
#ticks[1] = 2147483648
#ticks += get_ticks(0x62)
#ticks[2] -= 2147483648
#ticks += get_ticks(0x63)
#ticks[3] -= 2147483648
print(maxon1, maxon2, maxon3)

-48389 22759 31436


In [160]:
#IT IS JUST CONSTANTS CALCULATION IF YOU WANT TO ASSIGN CURRENT TICKS AS ZERO 
const1 = int(get_ticks(0x61)[0])-2147483648
const2 = int(get_ticks(0x62)[0])-2147483648
const3 = int(get_ticks(0x63)[0])-2147483648

In [62]:
ticks2 = ticks + ticks + ticks

In [49]:
a = int(ticks[1])

In [53]:
print(a,a)

2147483648 2147483648


In [50]:
a

2147483648

In [266]:
vx2: float = 0.4
print(len(bytearray(struct.pack("f", vx2))) )
print(bytearray(struct.pack("f", vx2)))
#print(size)

4
bytearray(b'\xcd\xcc\xcc>')


In [274]:
flt = get_test_float()
print(flt)

(0.5,)


In [277]:
flt = get_sent_test_float(0.5)
print(flt)

(0.5,)


In [289]:
flt = maxon_set_local_speed2(0.0,0.0,0)
print(flt)

(0.0,)


In [292]:
flt = maxon_set_local_speed2(0.0,0.0,0.2)
print(flt)

(0.0,)


In [293]:
flt = maxon_set_local_speed2(0.0,0.0,-0.2)
print(flt)

(0.0,)


In [312]:
while(1):
    flt = maxon_set_local_speed2(0.0,0.0,0.5)
    time.sleep(2)
    flt = maxon_set_local_speed2(0.0,0.0,-0.5)
    time.sleep(2)

KeyboardInterrupt: 

In [315]:
while(1):
    flt = maxon_set_local_speed2(0.1,0.0,0.0)
    time.sleep(2)
    flt = maxon_set_local_speed2(-0.1,0.0,0.0)
    time.sleep(2)

KeyboardInterrupt: 

In [7]:
while(1):
    flt = maxon_set_local_speed2(0.0,0.1,0.0)
    time.sleep(3)
    flt = maxon_set_local_speed2(0.0,-0.1,0.0)
    time.sleep(3)

NameError: name 'SET_LOCAL_SPEED_IN_MS' is not defined

In [368]:
v = 0.8
while(1):
    flt = maxon_set_local_speed2(v,0.0,0.0)
    time.sleep(2)
    flt = maxon_set_local_speed2(0.0,v,0.0)
    time.sleep(2)
    
    flt = maxon_set_local_speed2(-v,0.0,0.0)
    time.sleep(2)
    flt = maxon_set_local_speed2(0.0,-v,0.0)
    time.sleep(2)

SerialException: WriteFile failed (PermissionError(13, 'Устройство не опознает команду.', None, 22))

In [612]:
v = 0
flt = maxon_set_local_speed(0,0.0,0)

In [362]:
ser.flushInput()
speed = maxon_get_speed_in_ticks_by_id(0x11)
print(speed)
speed = maxon_get_speed_in_ticks_by_id(0x12)
print(speed)
speed = maxon_get_speed_in_ticks_by_id(0x13)
print(speed)

(-3914,)
(-3915,)
(-3914,)


In [628]:
maxon_enable_by_id(0x13)
maxon_enable_by_id(0x12)
maxon_enable_by_id(0x11)

In [620]:
maxon_disable_by_id(0x13)
maxon_disable_by_id(0x12)
maxon_disable_by_id(0x11)

In [398]:
x = get_global_coord_x()
print(x)

(-739.8231201171875,)


In [415]:
while(1):
    ser.flushInput()
    x = get_global_coord_x()
    print(x)
    time.sleep(0.2)

(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)
(0.36146312952041626,)


KeyboardInterrupt: 

In [422]:
while(1):
    btarr = bytearray([GET_GLOBAL_CORD_IN_M])
    ser.write(btarr)
    string = ser.read(12)
    print(struct.unpack('f', string[0:4]),struct.unpack('f', string[4:8]),struct.unpack('f', string[8:12]))
    #print(x)
    time.sleep(0.2)

(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)
(0.0012334136990830302,) (-0.00016468043031636626,) (-0.042808763682842255,)

KeyboardInterrupt: 

In [627]:
btarr = bytearray([GET_GLOBAL_CORD_IN_M])
ser.write(btarr)
string = ser.read(12)
print(struct.unpack('f', string[0:4]),struct.unpack('f', string[4:8]),struct.unpack('f', string[8:12]))

(-0.03498050197958946,) (0.03273211047053337,) (-1.4873161315917969,)


In [609]:
maxon_set_global_coord(0,0,0)

In [597]:
maxon_disable_by_id(0x13)
maxon_disable_by_id(0x12)
maxon_disable_by_id(0x11)
vx = 0.1
vy = 0
flt = maxon_set_local_speed(vx,vy,0)
time.sleep(4)
flt = maxon_set_local_speed(0,0.0,0)
maxon_enable_by_id(0x13)
maxon_enable_by_id(0x12)
maxon_enable_by_id(0x11)

In [464]:
maxon_enable_by_id(0x13)
maxon_enable_by_id(0x12)
maxon_enable_by_id(0x11)

In [724]:
ser.flushInput()
while(1):
    string = ser.read(4)
    ticks = struct.unpack('i', string)
    print(ticks)

(944531207,)
(944531207,)
(944531207,)
(944531207,)
(944530708,)
(944531207,)
(944531207,)


KeyboardInterrupt: 

In [407]:
ser.flushInput()
send_cmd(0x01)

In [44]:
while(1):
    maxon_set_local_speed(128,100,56)
    time.sleep(0.005)

KeyboardInterrupt: 

In [17]:
while(1):
    ser.flushInput()
    send_cmd(0x01)
    time.sleep(0.1)

KeyboardInterrupt: 

In [4]:
while(1):
    ser.flushInput()
    send_cmd(0x01)
    time.sleep(1)

KeyboardInterrupt: 

In [9]:
# ser.flushInput()
# send_cmd(0x01)
# time.sleep(0.01)
while(1):
    ser.flushInput()
    send_cmd(0x01)
    #ser.read(15)
    #time.sleep(0.1)
#print(ser.read(4))

KeyboardInterrupt: 

In [85]:
send_cmd(0x01)

NameError: name 'send_cmd' is not defined

In [119]:
ser.flushInput()
a = 110;
btarr = bytearray([0x01])+bytearray([a&0xFF])+bytearray([(a>>8)&0xFF])
ser.write(btarr)

3

In [46]:
angles = [300,300,200,100,100,100,100,100,100,100,100,0]
btarr = bytearray([0x01])
for ik in angles:
    btarr += bytearray([ik&0xFF])
    btarr += bytearray([(ik>>8)&0xFF])
ser.write(btarr)

25

In [81]:
packet = [0xFF,0xFF,0x05,0x02,0x05,0x15,0x25,0xB9]
btarr = bytearray()
for ik in packet:
    btarr += bytearray([ik])
    
ser.write(btarr)

8

In [130]:
packet = [0xFF,0xFF,0x05,0x02,0x05,0x15,0x25,0xB9]
btarr = bytearray()
for ik in packet:
    btarr += bytearray([ik])



while(1):
#     ser.flushInput()
    ser.write(btarr)
    string = ser.read(8)
#     time.sleep(0.1)
#     ticks = struct.unpack('i', string)
#     print(ticks)

KeyboardInterrupt: 

In [131]:
# packet = [0xFF,0xFF,0x05,0x02,0x05,0x15,0x25,0xB9]
btarr = bytearray([0xFF,0xFF,0x05,0x02,0x05,0x15,0x25,0xB9])
# for ik in packet:
#     btarr += bytearray([ik])



while(1):
#     ser.flushInput()
    ser.write(btarr)
    string = ser.read(8)
#     time.sleep(0.1)
#     ticks = struct.unpack('i', string)
#     print(ticks)

KeyboardInterrupt: 

In [124]:
packet = [0xFF,0xFF,0x05,0x02,0x05,0x15,0x25,0xB9]
btarr = bytearray()
for ik in packet:
    btarr += bytearray([ik])



while(1):
#     ser.flushInput()
    ser.write(btarr)
#     string = ser.read()
#     time.sleep(0.1)
#     ticks = struct.unpack('i', string)
#     print(ticks)

KeyboardInterrupt: 

In [ ]:
packet = [0xFF,0xFF,0x05,0x03,0x05,0x15,0x25,0xB9]
btarr = bytearray()
for ik in packet:
    btarr += bytearray([ik])
    
ser.write(btarr)

In [93]:
angles = 300
btarr = bytearray()

btarr += bytearray([angles&0xFF])
btarr += bytearray([(ik>>8)&0xFF])
ser.write(btarr)

NameError: name 'ik' is not defined

In [92]:
while(1):
    speed = robot_get_speed()
    print(speed)
    time.sleep(0.01)

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

KeyboardInterrupt: 

In [231]:
def test(a: string):
    a = "qwerty"
    

In [232]:
a1 = "kote"
print(a1)
test(a1)
print(a1)

kote
kote


In [ ]:
print(struct.unpack('f', string[0:4]),struct.unpack('f', string[4:8]),struct.unpack('f', string[8:12]))

In [2]:
def maxon_set_global_coord(x: float, y: float, p: float):
    btarr = bytearray([SET_GLOBAL_CORD_IN_M])
    btarr += bytearray(struct.pack("f", x))
    btarr += bytearray(struct.pack("f", y))
    btarr += bytearray(struct.pack("f", p))
    ser.write(btarr)

def get_global_coord_x():
    btarr = bytearray([GET_GLOBAL_CORD_IN_M])
    ser.write(btarr)
    string = ser.read(12)
    ticks = struct.unpack('f', string[4:8])
    return ticks

def maxon_set_local_speed2(vx: float, vy: float, p: float):
    btarr = bytearray([SET_LOCAL_SPEED_IN_MS]) + bytearray(struct.pack("f", vx)) + bytearray(struct.pack("f", vy)) + bytearray(struct.pack("f", p))
    ser.write(btarr)
    string = ser.read(4)
    ticks = struct.unpack('f', string)
    return ticks

def maxon_set_local_speed(vx: float, vy: float, p: float):
    btarr = bytearray([SET_LOCAL_SPEED_IN_MS])
    btarr += bytearray(struct.pack("f", vx))
    btarr += bytearray(struct.pack("f", vy))
    btarr += bytearray(struct.pack("f", p))
    ser.write(btarr)

def get_sent_test_float(flt_tst: float):
    btarr = bytearray([0x82]) + bytearray(struct.pack("f", flt_tst))
    ser.write(btarr)
    string = ser.read(4)
    ticks = struct.unpack('f', string)
    return ticks

def get_test_float():
    btarr = bytearray([0x81])
    ser.write(btarr)
    string = ser.read(4)
    ticks = struct.unpack('f', string)
    return ticks

def maxon_set_local_speed_old(vx: float, vy: float, p: float):
    #btarr = bytearray([SET_LOCAL_SPEED_IN_MS,vx&0xFF,(vx>>8)&0xFF,(vx>>16)&0xFF,(vx>>24)&0xFF, vy&0xFF,(vy>>8)&0xFF,(vy>>16)&0xFF,(vy>>24)&0xFF, p&0xFF,(p>>8)&0xFF,(p>>16)&0xFF,(p>>24)&0xFF])
    btarr = bytearray([SET_LOCAL_SPEED_IN_MS]) + bytearray(struct.pack("f", vx)) + bytearray(struct.pack("f", vy)) + bytearray(struct.pack("f", p))
    #ba = bytearray(struct.pack("f", vy)) + bytearray(struct.pack("f", vy))
    ser.write(btarr)
    #string = ser.read(2)
    #ticks = struct.unpack('h', string)
    #return ticks

def maxon_get_speed_in_ticks_by_id(id):
    btarr = bytearray([MAXON_GET_SPEED_IN_TICKS_BY_ID,id])
    ser.write(btarr)
    string = ser.read(2)
    ticks = struct.unpack('h', string)
    return ticks

def maxon_set_pwm_by_id(id,pwm):
    pwm2 = int(pwm)
    btarr = bytearray([MAXON_SET_PWM_BY_ID,id,pwm2&0xFF,(pwm2>>8)&0xFF])
    #btarr = bytearray([MAXON_SET_PWM_BY_ID,id,0x7F,0xFF])
    ser.write(btarr)

def maxon_enable_by_id(id):
    btarr = bytearray([MAXON_ENABLE_BY_ID,id])
    ser.write(btarr)
    #string = ser.read(2)
    #angle =struct.unpack('H', string)
    #return angle

def maxon_disable_by_id(id):
    btarr = bytearray([MAXON_DISABLE_BY_ID,id])
    ser.write(btarr)
    #string = ser.read(2)
    #angle =struct.unpack('H', string)
    #return angle

def send_cmd(cmd):
    btarr = bytearray([cmd])
    ser.write(btarr)
    #string = ser.read(2)
    #angle =struct.unpack('H', string)
    #return angle

def get_dyn_angle_by_id(id):
    btarr = bytearray([0x67,id])
    ser.write(btarr)
    string = ser.read(2)
    angle =struct.unpack('H', string)
    return angle

def get_ticks(cmd):
    btarr = bytearray([cmd])
    ser.write(btarr)
    string = ser.read(4)
    ticks =struct.unpack('L', string)
    return ticks

def tick2rad(ticks):
    global L
    global R
    global gearhead
    global encoder_ticks
    rads = ticks*2*numpy.pi/(encoder_ticks*gearhead);
    return rads

In [6]:
ticks = get_ticks()
wheel_rads = tick2rad(ticks)
forward_matrix = 1/R * numpy.matrix([ [-numpy.cos(numpy.radians(30)),  -numpy.cos(numpy.radians(60)),  -L], \
                                      [                            0,                              1,  -L], \
                                      [ numpy.cos(numpy.radians(30)),  -numpy.cos(numpy.radians(60)),  -L]])
inverse_matrix = numpy.linalg.inv(forward_matrix)
global_coords = numpy.matmul(inverse_matrix, numpy.transpose(wheel_rads))
print(global_coords.round(3))


TypeError: can't multiply sequence by non-int of type 'float'